In [2]:
from lxml import etree as ET
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import pandas as pd
import csv  
import itertools
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oluwayetty1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
new_df = pd.read_csv('/Users/oluwayetty1/Desktop/School/1-2/NLP/public_homework2/data/sample.csv') 
new_df

,lemma,neighbours,bbNID
0,Mr,"""Finally President Mr""",bn:00055346n
1,President,"""Finally Mr Mr Santer""",bn:00064234n
2,Mr,"""Mr President Santer among""",bn:00055346n
3,speak,"""among others of taking""",bn:00090943v
4,policy,"""at institutional""",bn:00063330n


In [66]:
bbNIDs = [x.replace(' ','') for x in new_df['bbNID'].values]
bbNIDs

['bn:00055346n',
 'bn:00064234n',
 'bn:00055346n',
 'bn:00090943v',
 'bn:00063330n']

In [78]:
data = pd.read_csv('/Users/oluwayetty1/Desktop/School/1-2/NLP/public_homework2/resources/bn2wn_mapping.txt', header=None)
babelnet = []
for d in data[0]:
    babelnet.append(d.split('\t')[0])
    
# babelnet

In [79]:
matching_ids = set(bbNIDs) - set(babelnet)
matching_ids

set()

In [15]:
x_features = new_df['neighbours']
x_features

0          "Finally President Mr"
1          "Finally Mr Mr Santer"
2     "Mr President Santer among"
3        "among others of taking"
4              "at institutional"
Name: neighbours, dtype: object

In [160]:
new_df['target'] = new_df['lemma'] + '_' + new_df['bbNID']
y = new_df['target'].str.replace(' ', '')
y

0           Mr_bn:00055346n
1    President_bn:00064234n
2           Mr_bn:00055346n
3        speak_bn:00090943v
4       policy_bn:00063330n
Name: target, dtype: object

In [10]:
def tokenize(features):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(features)
    corpus = tokenizer.texts_to_sequences(features)
    V = len(tokenizer.word_index) + 1
    return corpus, V

In [16]:
feature_corpus, feature_V = tokenize(x_features)
print(feature_V)
print(feature_corpus)
maxlen = max(max(feature_corpus, key=len))
print(maxlen)

11
[[2, 3, 1], [2, 1, 1, 4], [1, 3, 4, 5], [5, 6, 7, 8], [9, 10]]
4


In [19]:
target_corpus, target_V = tokenize(y)
print(target_V)
print(target_corpus)

10
[[2, 1, 3], [4, 1, 5], [2, 1, 3], [6, 1, 7], [8, 1, 9]]


In [155]:
# data_corpus = [feature_corpus[0]] + [target_corpus[0]]
# data_corpus


data_corpus = [[2, 3, 1], [2]], [[2, 1, 1, 4], [4]]
data_corpus

([[2, 3, 1], [2]], [[2, 1, 1, 4], [4]])

In [158]:
def generate_data(corpus, V):
    for words in corpus:
        contexts = []
        labels   = []            
        contexts.append(words[0])
        
#         print(words[1])

        x = sequence.pad_sequences(contexts, maxlen=maxlen)
        y = np_utils.to_categorical(words[1], V)
#         x = [[]]
#         print(y[0][0])
#         print(x)
        yield (x, y)

generate_data(data_corpus,target_V)


<generator object generate_data at 0x1321ddca8>

In [121]:
dim = 2

cbow = Sequential()
cbow.add(Embedding(input_dim=target_V, output_dim=dim, input_length=None))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(target_V, activation='softmax'))

In [122]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [159]:
for ite in range(10):
    loss = 0.
    for x, y in generate_data(data_corpus, target_V):
        loss += cbow.train_on_batch(x, y)
    print(ite, loss)

0 4.602357625961304
1 4.593607187271118
2 4.584865093231201
3 4.576032876968384
4 4.567131519317627
5 4.558171510696411
6 4.5491557121276855
7 4.5400848388671875
8 4.530958414077759
9 4.521773338317871


In [164]:
f = open('/Users/oluwayetty1/Desktop/School/1-2/NLP/public_homework2/vectors.vec' ,'w')
f.write('{} {}\n'.format(target_V-1, dim)) 

4

In [165]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

NameError: name 'tokenizer' is not defined